In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from string import digits
from collections import Counter
# from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
%matplotlib inline

/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/envs/sentiment/lib/python3.7/site-pack

In [2]:
PATH = ""
data_train = pd.read_csv(PATH+"dataset/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv(PATH+"dataset/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [3]:
data_train['Data']

0       Mình đã dùng anywhere thế hệ đầu, quả là đầy t...
1       Quan tâm nhất là độ trễ có cao không, dùng thi...
2       dag xài con cùi bắp 98k....pin trâu, mỗi tội đ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       Đang xài con m175 cùi mía , nhà xài nhiều chuộ...
                              ...                        
5095    Mình mua máy về đc 1 ngày mà điện thoại khác g...
5096    Có bạn nào dùng f1w ko.mình dùng m cảm thấy qu...
5097    Dùng oppo mà bộ nhớ 4gb thì k chơi games ...
5098    Sao tui thích xài hàng oppo mà lựa toàn mấy đứ...
5099    mới mở hộp ,oy mở vào camera mà đã có ảnh chụp...
Name: Data, Length: 5100, dtype: object

In [4]:
import bogo
import regex as re
import itertools
def map_to_unicode():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

def covert_unicode(txt):
    dicchar = map_to_unicode()
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
  
def preprocess(document):
    if type(document) is not str:
      document = str(document)
    # convert to lower case
    document = document.lower()

    #remove
  
    # remove html character
    document = re.sub(r'<[^>]*>', '', document)

    #convert all char to unicode
    document = covert_unicode(document)

    #remove adjacent identical characters
    document = ''.join(c[0] for c in itertools.groupby(document))

    #uwf=>ừ,....
    document = bogo.process_sequence(document)

    # remove error character
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)

    # remove multiple space character
    document = re.sub(r'\s+', ' ', document).strip()

    return document

In [5]:
data_train['Data'] = data_train['Data'].map(lambda x: preprocess(x))
data_train['Data']

0       mình đã dùng anywhere thế hệ đầu quả là đầy th...
1       quan tâm nhất là độ trễ có cao không dùng thi ...
2       dag xài con cùi bắp 98k pin trâu mỗi tội đánh ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       đang xài con m175 cùi mía nhà xài nhiều chuột ...
                              ...                        
5095    mình mua máy về đc 1 ngày mà điện thoại khác g...
5096    có bạn nào dùng f1ư ko mình dùng m cảm thấy qu...
5097    dùng ôp mà bộ nhớ 4gb thì k chơi games đc đâu ...
5098    sao tui thích xài hàng ôp mà lựa toàn mấy đứa ...
5099    mới mở hộp oy mở vào camera mà đã có ảnh chụp ...
Name: Data, Length: 5100, dtype: object

In [6]:
data_train.isnull().sum()

Class    0
Data     0
dtype: int64

In [7]:
data_train['Class'].value_counts()

-1    1700
 1    1700
 0    1700
Name: Class, dtype: int64

In [9]:
import json
import re
import itertools
import unidecode
import bogo

def read_file(file_path):
    fi = open(file_path, 'r', encoding='utf-8')
    ls = fi.readlines()
    return ls

'''load dictionary to mapping word'''
vowel_file = open(PATH+'correct_teencode_stopword/vietnamese_vowel.json', encoding='utf-8')
vowel_dic = json.load(vowel_file)

short_word_file = open(PATH+'correct_teencode_stopword/short_word.json', encoding='utf-8')
short_word_dic = json.load(short_word_file)

single_word_dic = read_file(PATH+'correct_teencode_stopword/unidecode_vietnamese_dic.txt')
single_word_dic = [re.sub('\n','', s) for s in single_word_dic]


def replace_one_one(word, dictionary):
    new_word = dictionary.get(word,word)
    return new_word
def correct_vowel(sent, vowel_dictionary):
    '''mapping a`, a\ --> à, ....'''
    words = sent.split()
    pattern = r'[aăâeêuưiyoôơ][.`~?\']'
    sent = ""
    for word in words:
        p = re.search(pattern, word)
        new_word = word
        if p:
            idx = p.span()
            replace_vowel = vowel_dictionary[word[idx[0]]][word[idx[0] + 1]]
            new_word = re.sub(pattern, replace_vowel, new_word)
        sent += new_word + ' '
    return sent

def correct_teencode(sent):
    sent = preprocess(sent)
    sent = correct_vowel(sent, vowel_dic)

    words = sent.split()
    sent = ""
    for word in words:
        new_word = ""
        if word[-1] in [',',';']:
            new_word = replace_one_one(word[:-1], short_word_dic)
            sent += new_word + word[-1]
        else:
            new_word = replace_one_one(word, short_word_dic)
            sent += new_word
        sent += ' '
    return sent[:-1]

In [10]:
data_fixteencode = data_train.copy()
data_fixteencode['Data'] = data_fixteencode['Data'].map(lambda x: correct_teencode(x))

In [11]:
data_fixteencode['Data']

0       mình đã dùng anywhere thế hệ đầu quả là đầy th...
1       quan tâm nhất là độ trễ có cao không dùng thi ...
2       dag xài con cùi bắp 98k pin trâu mỗi tội đánh ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       đang xài con m175 cùi mía nhà xài nhiều chuột ...
                              ...                        
5095    mình mua máy về được 1 ngày mà điện thoại khác...
5096    có bạn nào dùng f1ư không mình dùng m cảm thấy...
5097    dùng ôp mà bộ nhớ 4gb thì không chơi games đượ...
5098    sao tui thích xài hàng ôp mà lựa toàn mấy đứa ...
5099    mới mở hộp rồi mở vào camera mà đã có ảnh chụp...
Name: Data, Length: 5100, dtype: object

In [12]:
data_train = pd.concat([data_fixteencode, data_train])
data_train.shape

(10200, 2)

In [13]:
data_train = data_train.drop_duplicates()
data_train = data_train.dropna()
data_train['Class'].value_counts()

 0    2277
-1    2125
 1    2007
Name: Class, dtype: int64

In [14]:
data_train.isnull().sum()

Class    0
Data     0
dtype: int64

In [15]:
data_test['Data'] = data_test['Data'].map(lambda x: preprocess(x))
data_test['Data']

0       nói thiệt là mình thì thì chuột nào mình cũng ...
1       đang dùng mx1 cũng ngon nhưng chưa đầy năm mà ...
2       chưa thấy đc điểm thuyết phục để mua nhất là v...
3       những phần xem báo tra cứu bản đồ dịch vụ dùng...
4       đúng là mua ở việt nam không ứng dụng được gì ...
                              ...                        
1045                                              30 củ à
1046    aple bán dc thi samsung cũng lời nhiêu người k...
1047    có thể giúp android vượt trội so với ios chớ c...
1048    mẹ mình từng sang đài loan và có mua 1 cái iph...
1049    tùng minh nguyễn điện thoại của vk bị như này ...
Name: Data, Length: 1050, dtype: object

In [16]:
data_test['Class'].value_counts()

-1    350
 1    350
 0    350
Name: Class, dtype: int64

In [17]:
data_test['Data'] = data_test['Data'].map(lambda x: correct_teencode(x))
data_test['Data']

0       nói thiệt là mình thì thì chuột nào mình cũng ...
1       đang dùng mx1 cũng ngon nhưng chưa đầy năm mà ...
2       chưa thấy được điểm thuyết phục để mua nhất là...
3       những phần xem báo tra cứu bản đồ dịch vụ dùng...
4       đúng là mua ở việt nam không ứng dụng được gì ...
                              ...                        
1045                                              30 củ à
1046    aple bán được thi samsung cũng lời nhiêu người...
1047    có thể giúp android vượt trội so với ios chớ c...
1048    mẹ mình từng sang đài loan và có mua 1 cái iph...
1049    tùng minh nguyễn điện thoại của vợ bị như này ...
Name: Data, Length: 1050, dtype: object

In [18]:
data_test.isnull().sum()

Class    0
Data     0
dtype: int64

In [19]:
X_train = data_train['Data']
y_train = data_train['Class']
X_test = data_test['Data']
y_test = data_test['Class']

In [20]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidfconverter = TfidfVectorizer(max_features=10000)
X_train = tfidfconverter.fit_transform(X_train)

In [22]:
print("Training set size: {} - Test set size: {}".format(X_train.shape[0],X_test.shape[0]))
print("Vocabulary size: {}".format(len(tfidfconverter.vocabulary_)))

Training set size: 6409 - Test set size: 1050
Vocabulary size: 6333


In [23]:
from sklearn import svm

import time
start_time = time.time()

clf = svm.SVC()
clf.fit(X_train, y_train)

print('Training time: ', time.time() - start_time, 's.')

Training time:  11.719604015350342 s.


In [24]:
import time
start_time = time.time()

X_test = tfidfconverter.transform(X_test)

y_pred = clf.predict(X_test)
print('Prediction time is ', time.time() - start_time, 's.')

Prediction time is  1.2424347400665283 s.


In [25]:
from sklearn import metrics
print("Precision:\t", metrics.precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print("Recall:\t\t", metrics.recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print("F1:\t\t", metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print("Accuracy:\t", metrics.accuracy_score(y_test, y_pred))

Precision:	 0.7135851173523063
Recall:		 0.7076190476190476
F1:		 0.7088269357969691
Accuracy:	 0.7076190476190476


In [26]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.71      0.69      0.70       350
           0       0.64      0.73      0.68       350
           1       0.79      0.71      0.75       350

    accuracy                           0.71      1050
   macro avg       0.71      0.71      0.71      1050
weighted avg       0.71      0.71      0.71      1050

